In [27]:
# Packages for model learning
import numpy as np
import sklearn.metrics
import pandas as pd
import sklearn.model_selection

import torch
import torchvision
import gpytorch
import botorch

# Packages for data loading
import json
from pathlib import Path

In [28]:
# Packages for Jupyter notebook
import IPython.display as ipd
import matplotlib.pyplot as plt
import PIL
import base64

In [29]:
from functools import cache

tensor_to_image = torchvision.transforms.ToPILImage()
@cache
def tensor_to_url(tensor, size=128):
    return fr"data:image/png;base64,{base64.b64encode(PIL.ImageOps.contain(tensor_to_image(tensor), (size, size))._repr_png_()).decode('ascii')}"

# Load and process data

In [30]:
# Make all photos square
def pad_image(img):
    h,w = img.shape[1:]
    if h != w:
        new_w = max(h,w)
        pad_h, rem_h = divmod(new_w - h, 2)
        pad_w, rem_w = divmod(new_w - w, 2)
        padding = [pad_w, pad_h, pad_w+rem_w, pad_h+rem_h]
        return torchvision.transforms.functional.pad(img, padding, padding_mode='edge')
    return img

In [31]:
# Load dataset

# img_root = './dataset/dress_pure_renamed/'
# train_metadata = json.loads(Path('./dataset/dress_pure_renamed/train.json').read_text())
# test_metadata = json.loads(Path('./dataset/dress_pure_renamed/test.json').read_text())
# val_metadata = json.loads(Path('./dataset/dress_pure_renamed/val.json').read_text())

# class_labels = ['christian_dior', 'maison_margiela']
torch.manual_seed(0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = "cpu"

# img_root = Path('./dataset/')
# train_metadata = json.loads((img_root / 'train.json').read_text())
# test_metadata = json.loads((img_root / 'test.json').read_text())
# val_metadata = json.loads((img_root / 'val.json').read_text())
# img_root = "F://datasets//fashion_brands//"
img_root = "F://datasets//fashion_designers_list//"

print(img_root)

train_metadata = json.loads(Path(img_root + '\/train.json').read_text())
test_metadata = json.loads(Path(img_root + '\/test.json').read_text())
val_metadata = json.loads(Path(img_root + '\/val.json').read_text())

# class_labels = ["alexander_mcqueen","donatella_versace","karl_lagerfeld","yves_saint_laurent"]
class_labels = ["alexander_mcqueen", "donatella_versace", "john_galliano", "karl_lagerfeld", "yves_saint_laurent"]
class_labels = [x.replace('_',' ') for x in class_labels]
class_num = 5
image_size = 224

n_train = len(train_metadata)
n_test = len(test_metadata)
n_val = len(val_metadata)
n_all = n_train + n_test + n_val

all_classes = torch.empty(n_all, dtype=torch.int)
all_images = [None]*n_all
for i,meta in enumerate((*train_metadata, *test_metadata, *val_metadata)):
    if i%50==0:
        print(i)
    all_classes[i] = meta['label']
    all_images[i] = torchvision.io.read_image(str(Path(img_root + meta['file_path'])),mode=torchvision.io.ImageReadMode.RGB).to(device)
#     all_images[i] = pad_image(torchvision.transforms.functional.resize(all_images[i], image_size, antialias=True)) Dani
    all_images[i] = torchvision.transforms.functional.resize(pad_image(all_images[i]), image_size, antialias=True).to(device)

F://datasets//fashion_designers_list//
0


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700


In [32]:
n_classes = all_classes.max() + 1
print(n_classes)

tensor(5, dtype=torch.int32)


In [33]:
from resnet_model import resnet_model

In [34]:
resnet_extractor = resnet_model(class_num,backbone="resnet18").to(device)
# checkpoint = torch.load('results//fashion_brands.3layer.bsz_128sz_224.sgd0.002//best_model.pt', map_location=device)
checkpoint = torch.load('results//fashion_designers_c5.3layer.bsz_128sz_224.sgd0.002//best_model.pt', map_location=device)
resnet_extractor.load_state_dict(checkpoint['state_dict'])

E:\Users\nora\anaconda3\envs\deep_gp\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
E:\Users\nora\anaconda3\envs\deep_gp\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [35]:
for p in resnet_extractor.parameters():
    p.requires_grad = False

In [36]:
# Define resnet feature extractor
resnet_input_transform = torchvision.models.ResNet18_Weights.DEFAULT.transforms().to(device)
crop_size = resnet_input_transform.crop_size[0]
print(crop_size)

all_data = torch.empty(n_all, 3, crop_size, crop_size).to(device)
for i in range(n_all):
    # print("i="+str(i))
    # print(all_images[i].shape)
    all_data[i] = resnet_input_transform(all_images[i])

224


E:\Users\nora\anaconda3\envs\deep_gp\lib\site-packages\torchvision\transforms\functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [37]:
batch_size = 128
left = n_all%batch_size
batches_num = int(n_all/batch_size)
print(left)
if left!= 0:
    batches_num += 1
all_embeddings = []
# print(batches_num)
for i in range(1,batches_num+1):
    # print(i)
    if i == batches_num:
        all_embeddings.append(resnet_extractor.backbone(all_data[(batches_num-1)*batch_size:]))
    else:
        all_embeddings.append(resnet_extractor.backbone(all_data[(i-1)*batch_size:i*batch_size]))
    # print(len(all_embeddings))
all_embeddings = torch.cat(all_embeddings)

14


In [38]:
print(all_embeddings.shape)
print(n_all)
print(len(all_data))

torch.Size([2702, 512])
2702
2702


# tesing the renset

In [39]:
torch.manual_seed(0)
train_embeddings_res = all_embeddings[0:n_train]
testing_embeddings_res = all_embeddings[n_train:n_train+n_test]
train_pre_res = torch.nn.functional.softmax(resnet_extractor.fc(train_embeddings_res),dim=1)
testing_pre_res = torch.nn.functional.softmax(resnet_extractor.fc(testing_embeddings_res),dim=1)
train_pre_res_class = torch.argmax(train_pre_res,dim=1)
testing_pre_res_class = torch.argmax(testing_pre_res,dim=1)
#testing set
acc = sklearn.metrics.accuracy_score(all_classes[n_train:n_train+n_test], testing_pre_res_class.detach().cpu())
# auc = sklearn.metrics.roc_auc_score(all_classes[n_train:n_train+n_test],testing_pre_res.detach().cpu(),multi_class="ovo")
print(len(all_classes[n_train:n_train+n_test]))
print(len(testing_pre_res_class))
print(testing_pre_res_class.detach().cpu().numpy())

print("testing resnet acc:{:.2f}".format(acc))
# print("testing resnet auc:{:.2f}".format(auc))

# #training set
# acc = sklearn.metrics.accuracy_score(all_classes[0:n_train], train_pre_res_class.detach().cpu())
# auc = sklearn.metrics.roc_auc_score(all_classes[0:n_train],train_pre_res.detach().cpu(),multi_class="ovo")
# print("training resnet acc:{:.2f}".format(acc))
# print("training resnet auc:{:.2f}".format(auc))


541
541
[3 0 2 4 1 4 3 0 2 4 4 2 3 4 4 2 0 1 1 3 1 4 1 4 2 1 2 4 2 4 2 4 3 2 4 2 2
 4 4 4 4 3 1 0 1 3 4 4 1 1 1 2 0 0 1 2 4 1 2 4 3 4 3 2 1 3 2 3 2 2 2 2 2 4
 4 2 0 4 3 2 4 4 1 4 4 2 4 1 0 0 2 0 4 2 0 4 2 1 4 0 0 4 0 2 2 1 3 2 4 2 1
 0 4 4 4 2 4 2 4 2 4 4 4 4 0 4 4 4 3 4 1 0 0 2 2 3 4 2 2 4 0 4 4 2 0 2 2 2
 2 4 4 2 4 4 2 4 0 3 2 0 1 4 4 2 0 2 2 1 4 2 4 2 4 0 4 3 3 2 2 0 4 3 4 2 2
 4 2 4 3 1 1 0 2 4 0 4 4 4 0 4 4 2 2 0 4 4 1 1 4 0 4 3 0 4 4 4 2 4 4 4 0 4
 4 0 2 4 2 2 2 2 3 4 2 4 0 0 0 3 0 2 2 2 2 3 2 3 3 2 4 4 4 2 1 4 4 4 4 4 0
 2 4 2 4 1 4 4 3 2 1 4 3 3 4 1 2 4 4 4 1 2 4 0 4 3 4 4 1 4 4 4 4 1 4 3 2 0
 4 4 4 2 0 4 1 4 1 4 0 2 0 2 2 4 1 4 2 4 0 4 3 0 4 0 3 2 4 0 2 0 2 2 2 4 0
 3 2 4 4 3 0 1 4 4 4 1 4 2 0 3 2 4 1 1 4 2 2 0 2 4 0 3 0 3 0 1 4 4 4 2 0 4
 2 4 4 3 2 2 3 0 4 0 3 0 2 4 4 4 2 0 4 4 4 4 4 3 4 4 2 4 4 2 2 1 4 1 0 4 2
 0 2 4 2 2 4 3 3 3 4 2 4 0 2 4 4 4 4 4 4 4 2 4 4 4 4 2 2 3 2 1 1 4 4 2 1 4
 4 4 2 3 3 3 0 2 4 4 1 4 3 1 0 2 2 4 1 4 4 4 3 4 4 4 2 4 4 0 1 2 2 2 0 3 0
 4 1 4 2 4 0 4 0 

<!-- testing the resnet backbone -->

# Split train/test

In [40]:
kfold = sklearn.model_selection.KFold(5, shuffle=True, random_state=0)

In [41]:
idx_splits = list(kfold.split(np.empty(len(all_classes)), all_classes))
n_folds = len(idx_splits)

In [42]:
n_seed_rounds = 5

In [43]:
seed_round_index = pd.RangeIndex(n_seed_rounds, name='seed_i')
fold_index = pd.RangeIndex(n_folds, name='fold_i')

In [44]:
# latent_dim_index = pd.Index([2,5,10,12,15,20,30,40,50,60,70,80,90,100], name='dim_i')
latent_dim_index = pd.Index([2,5,10], name='dim_i')

# Define and train GP

In [45]:
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

class GPModel(ApproximateGP):
    def __init__(self, inducing_points,num_classes, latent_dim):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        # self.mean_module = gpytorch.means.ConstantMean()
        # self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

        self.mean_module = gpytorch.means.ConstantMean(batch_shape=torch.Size((num_classes,)))
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(batch_shape=torch.Size((num_classes,))),
            batch_shape=torch.Size((num_classes,)),
        )
        self.scaler = gpytorch.utils.grid.ScaleToBounds(-1,1)
        self.fc = torch.nn.Linear(inducing_points.shape[1],latent_dim)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def transform(self, x):
        x = self.fc(x)
        x = self.scaler(x)
        return x
    
    def forward(self, x):
        x = self.transform(x)
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [46]:
samples_built_gp_num = 500

In [47]:
from tqdm import tqdm

# Stochastic Variational GP Regression

In [58]:
gp_index = pd.MultiIndex.from_product([fold_index, seed_round_index, latent_dim_index])

optimization_trace = pd.Series([list() for _ in gp_index], index=gp_index)
models = pd.Series(index=gp_index, dtype=object)

for gp_i in gp_index:
    print(gp_i,'...', end=' ', flush=True)
    fold_i, seed_i, dim_i = gp_i
    train_idx, test_idx = idx_splits[fold_i]
    train_classes = all_classes[train_idx]
    train_embeddings = all_embeddings[train_idx]

    test_classes = all_classes[test_idx]
    test_embeddings = all_embeddings[test_idx]

    # initialize likelihood and model
    torch.manual_seed(seed_i)

    inducing_points = train_embeddings[:samples_built_gp_num, :]
    model = GPModel(inducing_points=inducing_points,num_classes=class_num,latent_dim=dim_i)
    # likelihood = gpytorch.likelihoods.GaussianLikelihood()
    likelihood = gpytorch.likelihoods.DirichletClassificationLikelihood(targets=train_classes)
    likelihood = likelihood.to(device)
    model = model.to(device)
    model.train()
    likelihood.train()

    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_classes.size(0))
    mll.train()

    optimizer = torch.optim.Adam(mll.parameters(), lr=0.01) 


    # creating dataloader
    from torch.utils.data import TensorDataset, DataLoader

    train_dataset = TensorDataset(train_embeddings, train_classes)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

    test_dataset = TensorDataset(test_embeddings, test_classes)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    num_epochs = 1
    epochs_iter = tqdm(range(num_epochs), desc="Epoch")
    for epoch in epochs_iter:
        minibatch_iter = tqdm(train_loader, desc="Minibatch", leave=False)
        for x_batch, y_batch in minibatch_iter:
            optimizer.zero_grad()
            x_batch = x_batch.to(device)
            y_batch = y_batch.to(device)
            output = likelihood(model(x_batch))
            loss = -mll(output, y_batch)
            loss = loss.sum()
            minibatch_iter.set_postfix(loss=loss.item())
            loss.backward()
            optimizer.step()
            
    models[gp_i] = model
    print('Done', flush=True)
    # botorch.optim.fit.fit_gpytorch_mll_scipy(mll, callback=lambda _,i: optimization_trace.append(i))

(0, 0, 2) ... 

Epoch:   0%|                                                                                     | 0/1 [00:00<?, ?it/s]E:\Users\nora\anaconda3\envs\deep_gp\lib\site-packages\gpytorch\likelihoods\gaussian_likelihood.py:300: GPInputWarning: You have passed data through a FixedNoiseGaussianLikelihood that did not match the size of the fixed noise, *and* you did not specify noise. This is treated as a no-op.
  warnings.warn(


# DirichletClassificationLikelihood

In [49]:
# gp_index = pd.MultiIndex.from_product([fold_index, seed_round_index, latent_dim_index])

# optimization_trace = pd.Series([list() for _ in gp_index], index=gp_index)
# models = pd.Series(index=gp_index, dtype=object)

# for gp_i in gp_index:
#     print(gp_i,'...', end=' ', flush=True)
#     fold_i, seed_i, dim_i = gp_i
#     train_idx, test_idx = idx_splits[fold_i]
#     train_classes = all_classes[train_idx]
#     train_embeddings = all_embeddings[train_idx]

#     test_classes = all_classes[test_idx]
#     test_embeddings = all_embeddings[test_idx]

#     # initialize likelihood and model
#     torch.manual_seed(seed_i)

#     inducing_points = train_embeddings[:samples_built_gp_num, :]
#     model = GPModel(inducing_points=inducing_points,num_classes=class_num,latent_dim=dim_i)
#     likelihood = gpytorch.likelihoods.GaussianLikelihood()
#     likelihood = likelihood.to(device)
#     model = model.to(device)
#     model.train()
#     likelihood.train()

#     mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_classes.size(0))
#     mll.train()

#     optimizer = torch.optim.Adam([
#     {'params': model.parameters()},
#     {'params': likelihood.parameters()},
#     ], lr=0.01) 


#     # creating dataloader
#     from torch.utils.data import TensorDataset, DataLoader

#     train_dataset = TensorDataset(train_embeddings, train_classes)
#     train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)

#     test_dataset = TensorDataset(test_embeddings, test_classes)
#     test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#     num_epochs = 1
#     epochs_iter = tqdm(range(num_epochs), desc="Epoch")
#     for epoch in epochs_iter:
#         minibatch_iter = tqdm(train_loader, desc="Minibatch", leave=False)
#         for x_batch, y_batch in minibatch_iter:
#             optimizer.zero_grad()
#             x_batch = x_batch.to(device)
#             y_batch = y_batch.to(device)
#             output = model(x_batch)

#             pred_samples = output.sample(torch.Size((256,))).exp()
#             probabilities = (pred_samples / pred_samples.sum(1, keepdim=True)).mean(0).detach().cpu()
#             pred_class = torch.argmax(probabilities,dim=0,keepdim=False)

#             loss = -mll(pred_class, y_batch)
#             minibatch_iter.set_postfix(loss=loss.item())
#             loss.backward()
#             optimizer.step()
            
#     models[gp_i] = model
#     print('Done', flush=True)
#     # botorch.optim.fit.fit_gpytorch_mll_scipy(mll, callback=lambda _,i: optimization_trace.append(i))

In [55]:
# f,axs = plt.subplots(n_folds,1,constrained_layout=True,figsize=(10,2*n_folds), sharex=True)
# for fold_i,ax in zip(range(n_folds), axs):
#     for gp_i in gp_index[gp_index.get_locs(pd.IndexSlice[fold_i,:,:])]:
#         ax.plot([r.step for r in optimization_trace[gp_i]], [r.fval for r in optimization_trace[gp_i]])
#     ax.set_ylabel('Marginal log-lik');
# axs.flat[-1].set_xlabel('Steps');
# plt.show(f);plt.close(f)
# # print(optimization_trace[-1].status, optimization_trace[-1].message)

## Metrics

In [51]:
metrics = pd.DataFrame(columns=['acc','auc'],index=gp_index)

In [52]:
n_plots = n_folds
n_cols = 5
n_rows = int(np.ceil(n_plots / n_cols))
# f,axs = plt.subplots(n_rows,n_cols, sharey=True, sharex=True, figsize=(n_cols*4, n_rows*4))
for fold_i in fold_index:
    train_idx, test_idx = idx_splits[fold_i]
    test_classes = all_classes[test_idx]
    test_embeddings = all_embeddings[test_idx]
    for gp_i in gp_index[gp_index.get_locs(pd.IndexSlice[fold_i,:,:])]:
        model = models[gp_i]
        model.eval()
        with torch.no_grad():
            test_pred_dist = model(test_embeddings)

        # test_pred_samples = test_pred_dist.sample(torch.Size((256,))).exp()
        # test_probabilities = (test_pred_samples / test_pred_samples.sum(1, keepdim=True)).mean(0).detach().cpu()

        test_pred_class = torch.argmax(test_pred_dist.loc,dim=0)
        # test_pred_class = torch.argmax(test_probabilities,dim=0,keepdim=False)

        acc = sklearn.metrics.accuracy_score(test_classes.cpu(), test_pred_class.cpu())
        # auc = sklearn.metrics.roc_auc_score(test_classes,test_probabilities,multi_class="ovo")
        metrics.at[gp_i, 'acc'] = acc
        # metrics.at[gp_i, 'auc'] = auc

        # disp = sklearn.metrics.ConfusionMatrixDisplay(
        #     sklearn.metrics.confusion_matrix(test_classes, test_pred_class),
        #     display_labels=class_labels
        # )
        # axs.flat[fold_i].set_title(f'Fold {fold_i+1:02d} (ACC: {acc*100:0.2f} AUC: {auc:0.2f})')
        # disp.plot(ax=axs.flat[fold_i], colorbar=False);

In [53]:
print('Metrics')
display(metrics.apply(lambda y: f'{y.mean():0.2f} ± {2*y.std():0.2f}').to_frame().T)
print('Metrics by latent dimension')
display(metrics.groupby(level=[2]).apply(lambda x: x.apply(lambda y: f'{y.mean():0.2f} ± {2*y.std():0.2f}')))

Metrics


,acc,auc
0,0.18 ± 0.03,nan ± nan


Metrics by latent dimension


,acc,auc
dim_i,,
2,0.18 ± 0.03,nan ± nan
5,0.18 ± 0.03,nan ± nan
10,0.18 ± 0.03,nan ± nan


# DirichletClassificationLikelihood